In [3]:
#Final Code
import numpy as np
import pandas as pd

# Load movie data (assuming 'movies.csv' has 'movieId', 'title', 'genres')
df = pd.read_csv('movie.csv')

# Take first 100 movies for simplicity
movies_data = df.head(100)[['movieId', 'title', 'genres']].copy()
# Convert genres into lists
movies_data['genres'] = movies_data['genres'].apply(lambda x: x.split('|'))

# Create movie index mapping
movie_index = {title: idx for idx, title in enumerate(movies_data['title'])}
index_movie = {idx: title for idx, title in enumerate(movies_data['title'])}

num_movies = len(movies_data)

# Initialize transition probability matrix P
P = np.zeros((num_movies, num_movies))

# Compute transition probabilities based on genre overlap
for i in range(num_movies):
    for j in range(num_movies):
        if i != j:
            shared_genres = len(set(movies_data.iloc[i]['genres']) & set(movies_data.iloc[j]['genres']))
            total_genres = len(set(movies_data.iloc[i]['genres']) | set(movies_data.iloc[j]['genres']))
            P[i, j] = shared_genres / total_genres if total_genres > 0 else 0

# Normalize P (so each row sums to 1)
row_sums = P.sum(axis=1, keepdims=True)
P = np.divide(P, row_sums, where=(row_sums != 0))  # Avoid division by zero

# Define Reward Function (R) - Higher reward for movies with more genre matches
R = np.zeros(num_movies)

# Value Iteration Parameters
gamma = 0.85  # Discount factor
theta = 1e-6  # Convergence threshold

# Initialize Value function
V = np.zeros(num_movies)

# Value Iteration Algorithm
while True:
    delta = 0
    new_V = np.zeros(num_movies)
    
    for s in range(num_movies):
        action_values = P[s] @ (R + gamma * V)  # Compute expected value
        sorted_actions = np.argsort(P[s])[::-1]  # Rank movies by transition probability
        
        best_actions = sorted_actions[:3]  # Top 3 recommendations
        new_V[s] = action_values if len(best_actions) > 0 else 0
        
        delta = max(delta, abs(new_V[s] - V[s]))  # Check for convergence
    
    V = new_V  # Update values
    
    if delta < theta:
        break  # Converged!

# Function to Recommend Movies
def recommend_movies(movie_name):
    if movie_name not in movie_index:
        print(f" Movie '{movie_name}' not found in database.")
        return []
    
    movie_idx = movie_index[movie_name]
    sorted_actions = np.argsort(P[movie_idx])[::-1]  # Rank movies
    
    recommendations = [index_movie[idx] for idx in sorted_actions[:3] if idx != movie_idx]
    
    return recommendations if recommendations else ["No similar movies found!"]

# User Input Loop
while True:
    user_movie = input("\n Enter a movie name (or type 'exit' to quit):  ").strip()
    
    if user_movie.lower() == 'exit':
        print("\n Thanks for using the movie recommender! Have a great day! ")
        break

    recommended_movies = recommend_movies(user_movie)

    print(f"\n If you liked '{user_movie}', you might enjoy:")
    for idx, movie in enumerate(recommended_movies, 1):
        print(f"   {idx}.  {movie}")



 Enter a movie name (or type 'exit' to quit):   Four Rooms (1995)



 If you liked 'Four Rooms (1995)', you might enjoy:
   1.  Ace Ventura: When Nature Calls (1995)
   2.  Friday (1995)
   3.  Father of the Bride Part II (1995)



 Enter a movie name (or type 'exit' to quit):   Friday (1995)



 If you liked 'Friday (1995)', you might enjoy:
   1.  Black Sheep (1996)
   2.  Bio-Dome (1996)
   3.  Father of the Bride Part II (1995)



 Enter a movie name (or type 'exit' to quit):   Bio-Dome (1996)


...


 If you liked 'Ace Ventura: When Nature Calls (1995)', you might enjoy:
   1.  Friday (1995)
   2.  Four Rooms (1995)
   3.  Father of the Bride Part II (1995)



 Enter a movie name (or type 'exit' to quit):   Father of the Bride Part II (1995)



 If you liked 'Father of the Bride Part II (1995)', you might enjoy:
   1.  Ace Ventura: When Nature Calls (1995)
   2.  Four Rooms (1995)
   3.  Friday (1995)



 Enter a movie name (or type 'exit' to quit):   exit



 Thanks for using the movie recommender! Have a great day! 
